# Confision Matrix

In [1]:
%load_ext autoreload
%autoreload 2

## Imports

In [2]:
from deepchecks.vision.dataset import VisionData
from torchvision import models
import torchvision
import torch
from torch import nn
from torchvision.transforms import ToTensor
import copy

In [3]:
from deepchecks.vision.checks.performance import ConfusionMatrixReport

In [11]:

from deepchecks.vision.datasets.classification import mnist
def simple_nn():
    torch.manual_seed(42)
    class NeuralNetwork(nn.Module):
        def __init__(self):
            super(NeuralNetwork, self).__init__()
            self.flatten = nn.Flatten()
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(28 * 28, 512),
                nn.ReLU(),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, 10)
            )

        def forward(self, x):
            x = self.flatten(x)
            logits = self.linear_relu_stack(x)
            return logits

    model = NeuralNetwork().to('cpu')
    return model

In [12]:
def trained_mnist(simple_nn, mnist_data_loader_train):
    torch.manual_seed(42)
    simple_nn = copy.deepcopy(simple_nn)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(simple_nn.parameters(), lr=1e-3)
    size = len(mnist_data_loader_train.dataset)
    # Training 1 epoch
    simple_nn.train()
    for batch, (X, y) in enumerate(mnist_data_loader_train):
        X, y = X.to('cpu'), y.to('cpu')

        # Compute prediction error
        pred = simple_nn(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    return simple_nn


In [29]:
train_ds = mnist.load_dataset(train=True, object_type='Dataset')
test_ds = mnist.load_dataset(train=False, object_type='Dataset')
model = trained_mnist(simple_nn(), train_ds.get_data_loader())


loss: 2.287713  [    0/60000]
loss: 2.271768  [ 6400/60000]
loss: 2.227654  [12800/60000]
loss: 2.213990  [19200/60000]
loss: 2.159032  [25600/60000]
loss: 2.118144  [32000/60000]
loss: 2.020085  [38400/60000]
loss: 1.975659  [44800/60000]
loss: 1.823154  [51200/60000]
loss: 1.817346  [57600/60000]


In [30]:
from deepchecks.vision.utils.classification_formatters import ClassificationPredictionFormatter
check = ConfusionMatrixReport(prediction_formatter=ClassificationPredictionFormatter(nn.Softmax(dim=1)))
check.run(train_ds, model)

Confusion Matrix Calculate the confusion matrix of the model on the given dataset. Read More... Additional Outputs Showing 10 of 10 classes:

# confusion matrix report object detection

In [19]:
from deepchecks.vision.datasets.detection import coco
import numpy as np

In [20]:
yolo = coco.load_model(pretrained=True)


Using cache found in /Users/dbi/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-1-30 torch 1.10.1 CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [21]:
coco_dataloader = coco.load_dataset()

Using downloaded and verified file: /Users/dbi/git/deepchecks/deepchecks/vision/datasets/detection/coco128.zip
Extracting /Users/dbi/git/deepchecks/deepchecks/vision/datasets/detection/coco128.zip to /Users/dbi/git/deepchecks/deepchecks/vision/datasets/detection


In [22]:
from deepchecks.vision.dataset import VisionData

In [23]:
from deepchecks.vision.utils.detection_formatters import DetectionLabelFormatter, DetectionPredictionFormatter
train_ds = VisionData(coco_dataloader, label_transformer=DetectionLabelFormatter(lambda x: x), num_classes=80)

In [28]:
check = ConfusionMatrixReport(prediction_formatter=DetectionPredictionFormatter(coco.yolo_wrapper), categories_to_display=100)
check.run(train_ds, yolo)

/Users/dbi/git/deepchecks/venv/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Disabling

/Users/dbi/git/deepchecks/venv/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Disabling

/Users/dbi/git/deepchecks/venv/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Disabling



Confusion Matrix Calculate the confusion matrix of the model on the given dataset. Read More... Additional Outputs last category are detections that do not overlap with labeled data and labels that have not been detected. Showing 100 of 80 classes: